In [0]:
# =======================================================================
# PASSO 1: CONFIGURAÇÃO E CARGA DOS CEPS DE ORIGEM
# =======================================================================
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import DoubleType

print("Iniciando a criação da tabela de geolocalização fictícia.")

# Nome da tabela de onde vamos ler os CEPs
NOME_TABELA_ORIGEM = "production.transient.pms_hoteis"

try:
    # Carrega a tabela de hóspedes e seleciona apenas os CEPs únicos e não nulos
    df_ceps = spark.table(NOME_TABELA_ORIGEM) \
        .select("cep") \
        .filter(F.col("cep").isNotNull()) \
        .distinct()
    
    print(f"Encontrados {df_ceps.count()} CEPs únicos na tabela '{NOME_TABELA_ORIGEM}'.")

except Exception as e:
    print(f"Erro ao carregar a tabela '{NOME_TABELA_ORIGEM}': {e}")
    dbutils.notebook.exit("Verifique se a tabela de hóspedes existe antes de executar este script.")


# =======================================================================
# PASSO 2: GERAÇÃO ALEATÓRIA DE LATITUDE E LONGITUDE
# =======================================================================

# Limites geográficos aproximados do Brasil (bounding box)
LAT_MIN, LAT_MAX = -33.75, 5.27  # Do Sul ao Norte
LON_MIN, LON_MAX = -73.98, -34.79 # Do Oeste ao Leste

print("Gerando coordenadas aleatórias dentro dos limites do Brasil...")

# Usamos a função nativa F.rand() do Spark, que gera um número aleatório entre 0 e 1.
# Em seguida, ajustamos a escala desse número para que ele se encaixe no nosso intervalo de lat/lon.
# Esta abordagem é 100% executada no motor do Spark, sendo extremamente performática.
df_geolocalizacao_ficticia = df_ceps.withColumn(
    "latitude", 
    (F.rand() * (LAT_MAX - LAT_MIN) + LAT_MIN).cast(DoubleType())
).withColumn(
    "longitude", 
    (F.rand() * (LON_MAX - LON_MIN) + LON_MIN).cast(DoubleType())
)

print("Coordenadas fictícias geradas com sucesso.")
display(df_geolocalizacao_ficticia.limit(10))


# =======================================================================
# PASSO 3: SALVAR A TABELA DELTA FINAL
# =======================================================================

# Você pode alterar "nome_tabela" para o nome que preferir
NOME_TABELA_FINAL = "production.transient.api_geolocalizacao_ceps"

print(f"\nSalvando a tabela de referência em: {NOME_TABELA_FINAL}")

df_geolocalizacao_ficticia.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(NOME_TABELA_FINAL)

print("Tabela salva com sucesso!")

# Otimiza a tabela para consultas rápidas por CEP
print("Otimizando a tabela...")
spark.sql(f"OPTIMIZE {NOME_TABELA_FINAL} ZORDER BY (cep)")
print("Otimização concluída.")


# =======================================================================
# PASSO 4: VERIFICAÇÃO
# =======================================================================
df_verificacao = spark.table(NOME_TABELA_FINAL)
print(f"\nVerificação: a tabela '{NOME_TABELA_FINAL}' foi criada com {df_verificacao.count()} registros.")
display(df_verificacao.limit(5))